<a href="https://colab.research.google.com/github/tony-wade/Reverse-Engineering/blob/main/excel_to_bin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 如何判讀disassemble後的成品(assembly正確與否)

*   interrupt 和 RETI會常用(臨時作動用), delay則盡量避免(=停擺待機 vs 硬體觸發)
```
 對於8051 (按絕對位置 = 初始優先級排序):
  1.   0000  Reset(接著執行此指令)
  2.   0003  INT0 = External HW Interrupt 0
  3.   000B  TF0 = Timer 0 Interrupt
  4.   0013  INT1 = External HW Interrupt 1
  5.   001B  TF1 = Timer 1 Interrupt
  6.   0023  RI, TI = Serial Com Interrupt (讀or發送訊號時用)

    另有一暫存器:IE Register = hex code能控制上述各interrupt要作動與否
    MOV IE, #81H    ; 使能外部中斷 INT0
```
*  CALL系列=副程式, 結束時需用RET回分支點
*  JUMP系列=直接跳轉, 用於LOOP或分支
*  0000時多為JMP,或 MOV(較少)




https://ebooks.inflibnet.ac.in/csp13/chapter/interrupt-handling-and-assembly-programming/

In [ ]:
import zipfile
from google.colab import files

# 獲取目錄中所有的 .bin 檔案
bin_files = [file for file in os.listdir() if file.endswith('.bin')]

# 建立壓縮檔
with zipfile.ZipFile('bin_files.zip', 'w') as zipf:
    # 將所有 .bin 檔案添加到壓縮檔中
    for file in bin_files:
        zipf.write(file)

# 下載壓縮檔
files.download('bin_files.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
excel_to_bin(input_folder=input_folder)
# excel_to_bin(input_folder=input_folder, mode='bitwise_not')
# excel_to_bin(input_folder=input_folder, big_endian=False)
# excel_to_bin(input_folder=input_folder, mode='bitwise_not', big_endian=False)

Opening file: ./test.xlsx


In [ ]:
input_folder = '.'
remove_sequences = [
    ['d3', '5e', '6d', 'dd', 'b3', '2c', 'fb', 'ee'],
    ['b4', '57', '5b', 'f7', '2e'],
    ['eb', '35', 'ed', 'd5', 'd6', '7d', 'cb'],
    ['6a', '4d', '7b', 'b5', '75', 'ff', 'ba', 'ee'],
    ['d3', '5e', '6d', 'dd', 'bf', '2e', 'ba'],
    ['b4', '57', '5b', '77', 'ac', 'cb', 'be'],
    ['ed', 'd5', 'd6', '9d', 'eb'],
    ['4d', '7b', 'b5', '75', 'd7', 'ba']
]
target_sequence = ['a5', 'f1']

In [ ]:
def xor(data, hex_value):
    """
    XORs two hexadecimal values

    Args:
        data (str): The first hexadecimal value.
        hex_value (str): The second hexadecimal value.

    Returns:
        str: The result of XOR operation in uppercase string format.
    """
    return format((int(data, 16) ^ int(hex_value, 16)), '02X')


def extract_row_data(row_data, excess_hex_seq=None, target_hex_seq=None):
    """
    Extract specified sequences from row data and check for target sequence.

    Args:
        row_data (pd.Series): Row data to be processed.
        excess_hex_seq (list of lists): List of sequences to be removed if they match exactly.
        target_hex_seq (list): Target sequence to check for.

    Returns:
        list or None: List of extracted data if target sequence is found, otherwise None.
    """
    # Combine all non-null data in the row into a single string
    data_str = ' '.join(row_data.dropna().tolist())

    if excess_hex_seq:
        # Remove specified sequences from the data string
        for seq in excess_hex_seq:
            seq_str = ' '.join(seq)
            data_str = data_str.replace(seq_str, '')

    # Convert the data string back to a list
    result = data_str.split()

    if target_hex_seq:
        # Check for the presence of the target sequence
        target_seq_str = ' '.join(target_hex_seq)
        target_index = data_str.find(target_seq_str)
        if target_index != -1:
            # Extract data after the target sequence
            after_target_data = data_str[target_index + len(target_seq_str):].strip()
            result = after_target_data.split()
        else:
            result = []

    return result


def endian_conversion(byte_datas, big_endian):
    """
    Perform endian conversion on a list of byte data.

    Args:
        byte_datas (list): List of byte data.
        big_endian (bool): Flag indicating big endian or little endian.

    Returns:
        bytes: Endian-converted binary data.
    """
    return b''.join(byte_datas) if big_endian else b''.join(reversed(byte_datas))



def process_row_data(row_data, big_endian, xor_data=None):
    """
    Process row data to generate binary data.

    Args:
        row_data (pd.Series): Row data to be processed.
        big_endian (bool): Flag indicating big endian or little endian.
        xor_data (str): Optional XOR data for encryption.

    Returns:
        bytes: Processed binary data.
    """
    row = extract_row_data(row_data=row_data)  # 手動調整remove_seq, target_seq

    if row:
        if xor_data:
            byte_datas = [bytes([int(xor(byte, xor_data), 16)]) for byte in row]
        else:
            byte_datas = [bytes([int(byte, 16)]) for byte in row]

        return endian_conversion(byte_datas, big_endian)


def data_processor(row_datas, output_file, big_endian, mode=None, xor_data=None):
    """
    Process row data and write binary data to output file.

    Args:
        row_datas (pd.DataFrame): DataFrame containing row data.
        output_file (str): Output file path.
        big_endian (bool): Flag indicating big endian or little endian.
        mode (str): Optional mode for processing data.
        xor_data (str): Optional XOR data for encryption.
    """
    row_datas = row_datas.astype(str)
    with open(output_file, 'wb') as f:
        if mode == 'bitwise_not':
            xor_data = 'FF'
        for _, row_data in row_datas.iterrows():
            bin_data = process_row_data(row_data, big_endian, xor_data)
            f.write(bin_data)


def excel_to_bin(input_folder, mode=None, big_endian=True, xor_list=None):
    """
    Convert Excel files to binary files.

    Args:
        input_folder (str): Folder containing Excel files.
        mode (str): Optional mode for processing data.
        big_endian (bool): Flag indicating big endian or little endian.
        xor_list (list): List of XOR data values for encryption.
    """
    for filename in os.listdir(input_folder):
        if filename.endswith(".xlsx"):
            file_path = os.path.join(input_folder, filename)
            print("Opening file:", file_path)

            df = pd.read_excel(file_path, header=None, dtype=str)
            df.columns = range(df.shape[1])

            endian_str = "big" if big_endian else "small"
            if xor_list:
                for xor_value in xor_list:
                    output_file = "XOR_" + xor_value + "_" + os.path.splitext(filename)[0] + ".bin"
                    data_processor(row_data=df, output_file=output_file, big_endian=big_endian, xor_data=xor_value)
            else:
                if mode:
                    output_file = f"{mode}_{endian_str}_{os.path.splitext(filename)[0]}.bin"
                    data_processor(df, output_file, big_endian, mode)
                else:
                    output_file = f"{endian_str}_{os.path.splitext(filename)[0]}.bin"
                    data_processor(df, output_file, big_endian)

In [ ]:
import os
import pandas as pd